In [1]:
import sqlite3
import pandas as pd  

def run_query(database, query):
    
    conn = sqlite3.connect(database)
    
    try:
        df = pd.read_sql_query(query, conn)
    finally:
        conn.close()
    return df

database = "sqlite_db_pythonsqlite.db"

query1 = "SELECT * FROM FACILITIES"
df_facilities = run_query(database, query1)
df_facilities

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [2]:
#Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members! 
query_q10 = """
SELECT 
    f.name AS facility_name,
    SUM(
        CASE 
            WHEN b.memid = 0 THEN f.guestcost * b.slots  -- Revenue from guests
            ELSE f.membercost * b.slots                  -- Revenue from members
        END
    ) AS total_revenue
FROM 
    Facilities f
LEFT JOIN 
    Bookings b ON f.facid = b.facid
GROUP BY 
    f.name
HAVING 
    total_revenue < 1000
ORDER BY 
    total_revenue
"""
df_revenue = run_query(database, query_q10)
df_revenue

,facility_name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


In [3]:
#Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order
query_q11 = """ select m.surname, m.firstname, m.memid, m.recommendedby, r.firstname, r.surname
from Members m
left join Members r on r.memid = m.recommendedby
order by m.surname, m.firstname
"""
df_member = run_query(database, query_q11)
df_member

,surname,firstname,memid,recommendedby,firstname,surname
0,Bader,Florence,15,9,Ponder,Stibbons
1,Baker,Anne,12,9,Ponder,Stibbons
2,Baker,Timothy,16,13,Jemima,Farrell
3,Boothe,Tim,8,3,Tim,Rownam
4,Butters,Gerald,5,1,Darren,Smith
5,Coplin,Joan,22,16,Timothy,Baker
6,Crumpet,Erica,36,2,Tracy,Smith
7,Dare,Nancy,7,4,Janice,Joplette
8,Farrell,David,28,,None,None
9,Farrell,Jemima,13,,None,None


In [4]:
#Q12: Find the facilities with their usage by member, but not guests
query_q12 = """select distinct f.name from Facilities f 
left join bookings b on f.facid = b.facid
where b.memid != 0
"""
df_mem_used_fac = run_query(database, query_q12)
df_mem_used_fac

,name
0,Table Tennis
1,Massage Room 1
2,Snooker Table
3,Pool Table
4,Tennis Court 1
5,Squash Court
6,Badminton Court
7,Tennis Court 2
8,Massage Room 2


In [5]:
#Q13: Find the facilities usage by month, but not guests
query_q13 = """select f.name as facility_name, 
count(*) as usage_count, 
strftime('%Y-%m', b.starttime) as month_of_year
from Facilities f 
left join bookings b on f.facid = b.facid
where b.memid != 0
group by f.name, strftime('%Y-%m', b.starttime)
order by month_of_year
"""
df_mem_used_fac_m = run_query(database, query_q13)
df_mem_used_fac_m

,facility_name,usage_count,month_of_year
0,Badminton Court,51,2012-07
1,Massage Room 1,77,2012-07
2,Massage Room 2,4,2012-07
3,Pool Table,103,2012-07
4,Snooker Table,68,2012-07
5,Squash Court,23,2012-07
6,Table Tennis,48,2012-07
7,Tennis Court 1,65,2012-07
8,Tennis Court 2,41,2012-07
9,Badminton Court,132,2012-08
